In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time 
import os
import matplotlib.pyplot as plt
import base64
import concurrent.futures

def scrape_data(url):
    data = {
        'merchant' : [],
        'category' : [],
        'star' : [],
        'address' : [],
        'oper_time' : [],
        'starCount' : [],
        'reviewCount' : []
    }
    data2 = {
        'user_name' : [],
        'user_rank' : [],
        'num_response' : [],
        'user_star' : [], 
        'time' : [],
        'rating' : [],
        'content' : []
    }

    driver = webdriver.Chrome()
    driver.get(url)
    driver.implicitly_wait(3)

    # 영업시간 더 보기 클릭
    try:
        oper_more = driver.find_elements(By.CSS_SELECTOR, 'a[data-logevent = "main_info,more_timeinfo"]')
        oper_more[0].click()
        time.sleep(2)  

    except:
        pass

    # 후기 더보기 클릭
    try:
        link_more = driver.find_element(By.CLASS_NAME, 'txt_more')
        while link_more.text == "후기 더보기":
            link_more = driver.find_element(By.CLASS_NAME, 'txt_more')
            if link_more.text != "후기 접기":
                link_more.click()
                time.sleep(0.3)
    except:
        pass
    time.sleep(0.1)
  
    soup = bs(driver.page_source, 'html.parser')

    test_image = driver.execute_script("return document.querySelectorAll('canvas')[1].toDataURL();")
    image_data = test_image.split(',')[1]
    image_data_decoded = base64.b64decode(image_data)

    test_image2 = driver.execute_script("return document.querySelectorAll('canvas')[2].toDataURL();")
    image_data2 = test_image2.split(',')[1]
    image_data_decoded2 = base64.b64decode(image_data2)

    driver.close()

    mer = soup.find('h2', {'class':'tit_location'}).get_text()
    data['merchant'].append(mer)
    data['category'].append(soup.find('span',{'class':'txt_location'}).get_text().split(':')[1].strip())
    data['address'].append(soup.find('span', {'class': 'txt_address'}).get_text().replace(" ", "").replace("\n", ""))

    if len(soup.find_all('span',{'class':'color_b'})) > 1:
        data['star'].append(soup.find_all('span', {'class':'color_b'})[0].get_text().strip('점'))
        data['starCount'].append(soup.find('a', {'class': 'link_evaluation'})['data-cnt'])
        data['reviewCount'].append(soup.find_all('span', {'class':'color_b'})[1].get_text().strip('개'))
    else:
        data['star'].append('0')
        data['starCount'].append('0')
        data['reviewCount'].append(soup.find('span', {'class':'color_b'}).get_text().strip('개'))

    if oper_more:    
        data['oper_time'].append(soup.find_all('ul', {'class':'list_operation'})[1].get_text().strip())
    else:
        data['oper_time'].append(soup.find('ul', {'class':'list_operation'}).get_text().strip().replace("\n", ""))

    reveiw_listed = soup.find_all('li', {'data-ismy':'false'})

    for j in range(len(reveiw_listed)):
        data2['user_name'].append(soup.find_all('span',{'class':'txt_username'})[j].get_text())
        data2['user_rank'].append(soup.find_all('span', {'class' : 'txt_badge'})[j].get_text().strip('레벨'))
        data2['num_response'].append(
            soup.find_all('div',{'class':'unit_info'})[j].find_all('span',{'class':'txt_desc'})[0].get_text()
        )
        data2['user_star'].append(
            soup.find_all('div',{'class':'unit_info'})[j].find_all('span',{'class':'txt_desc'})[1].get_text()
        )
        data2['time'].append(
            soup.find_all('div',{'class':'unit_info'})[j].find('span',{'class':'time_write'}).get_text()
        )
        data2['content'].append(
            soup.find_all('p', {'class':'txt_comment'})[j].get_text().strip('더보기')
        )

        style_attribute = soup.find_all('div', {'class':'star_info'})[j].find('span',{'class':'ico_star inner_star'})['style']
        width_percent = float(style_attribute.split(':')[1].strip('%;'))
        rating_out_of_five = width_percent / 20  
        data2['rating'].append(rating_out_of_five)

    df2 = pd.DataFrame(data2)
    df2.to_csv(f'./data2/{mer}.csv', index=False)

    with open(f'./image/{mer}1.png', 'wb') as f:
        f.write(image_data_decoded)

    with open(f'./image/{mer}2.png', 'wb') as f:
        f.write(image_data_decoded2)

    print(f"{mer}.csv 및 이미지 저장 완료")

    return data

def main():
    driver = webdriver.Chrome()
    driver.get('https://map.kakao.com/')
    time.sleep(2)

    searchbox = driver.find_element(By.ID, 'search.keyword.query')
    searchbox.send_keys('홍대 맛집'+ Keys.ENTER)
    time.sleep(3)

    list_url = []

    driver.find_element(By.ID, 'info.search.place.more').click()
    time.sleep(2)
    driver.find_element(By.ID, 'info.search.page.no1').click()
    time.sleep(2)

    for count in range(1):
        for inx in range(2, 6):
            moreview_element = driver.find_elements(By.CSS_SELECTOR, 'a[data-id="moreview"]')
            for i in range(len(moreview_element)):
                list_url.append(moreview_element[i].get_attribute('href'))
            driver.find_element(By.ID, f'info.search.page.no{inx}').click()
            if inx == 5:
                moreview_element = driver.find_elements(By.CSS_SELECTOR, 'a[data-id="moreview"]')
                for i in range(len(moreview_element)):
                    list_url.append(moreview_element[i].get_attribute('href'))
            time.sleep(2)
        driver.find_element(By.ID ,'info.search.page.next').click()
        time.sleep(2)

    driver.close()

    data_list = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(scrape_data, list_url)
        for result in results:
            data_list.append(result)

    df = pd.DataFrame(data_list)
    df.to_csv("./data1/data.csv", index=False)

if __name__ == "__main__":
    main()


In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time 
import os
import matplotlib.pyplot as plt
import base64
import concurrent.futures


def scrape_data(url):
    data = {
        'merchant' : [],
        'category' : [],
        'star' : [],
        'address' : [],
        'oper_time' : [],
        'starCount' : [],
        'reviewCount' : []
    }
    data2 = {
        'user_name' : [],
        'user_rank' : [],
        'num_response' : [],
        'user_star' : [], 
        'time' : [],
        'rating' : [],
        'content' : []
    }

    driver = webdriver.Chrome()
    driver.get(url)
    driver.implicitly_wait(3)

    # 영업시간 더 보기 클릭
    try:
        oper_more = driver.find_elements(By.CSS_SELECTOR, 'a[data-logevent = "main_info,more_timeinfo"]')
        oper_more[0].click()
        time.sleep(2)  

    except:
        pass

    # 후기 더보기 클릭
    try:
        link_more = driver.find_element(By.CLASS_NAME, 'txt_more')
        while link_more.text == "후기 더보기":
            link_more = driver.find_element(By.CLASS_NAME, 'txt_more')
            if link_more.text != "후기 접기":
                link_more.click()
                time.sleep(0.3)
    except:
        pass
    time.sleep(0.1)
  
    soup = bs(driver.page_source, 'html.parser')

    try:
        test_image = driver.execute_script("return document.querySelectorAll('canvas')[1].toDataURL();")
        image_data = test_image.split(',')[1]
        image_data_decoded = base64.b64decode(image_data)

        test_image2 = driver.execute_script("return document.querySelectorAll('canvas')[2].toDataURL();")
        image_data2 = test_image2.split(',')[1]
        image_data_decoded2 = base64.b64decode(image_data2)
    except:
        pass

    try:    
        print(driver.find_elements(By.CSS_SELECTOR, 'h2[class = "tit_location"]')[1].text,"파싱완료")    
    except:
        print('예외')
    driver.close()


    mer = soup.find('h2', {'class':'tit_location'}).get_text()
    data['merchant'].append(mer)
    data['category'].append(soup.find('span',{'class':'txt_location'}).get_text().split(':')[1].strip())
    data['address'].append(soup.find('span', {'class': 'txt_address'}).get_text().replace(" ", "").replace("\n", ""))

    if len(soup.find_all('span',{'class':'color_b'})) > 1:
        data['star'].append(soup.find_all('span', {'class':'color_b'})[0].get_text().strip('점'))
        data['starCount'].append(soup.find('a', {'class': 'link_evaluation'})['data-cnt'])
        data['reviewCount'].append(soup.find_all('span', {'class':'color_b'})[1].get_text().strip('개'))
    else:
        data['star'].append('0')
        data['starCount'].append('0')
        data['reviewCount'].append(soup.find('span', {'class':'color_b'}).get_text().strip('개'))

    if oper_more:    
        data['oper_time'].append(soup.find_all('ul', {'class':'list_operation'})[1].get_text().strip())
    else:
        data['oper_time'].append(soup.find('ul', {'class':'list_operation'}).get_text().strip().replace("\n", ""))

    reveiw_listed = soup.find_all('li', {'data-ismy':'false'})

    for j in range(len(reveiw_listed)):
        data2['user_name'].append(soup.find_all('span',{'class':'txt_username'})[j].get_text())
        data2['user_rank'].append(soup.find_all('span', {'class' : 'txt_badge'})[j].get_text().strip('레벨'))
        data2['num_response'].append(
            soup.find_all('div',{'class':'unit_info'})[j].find_all('span',{'class':'txt_desc'})[0].get_text()
        )
        data2['user_star'].append(
            soup.find_all('div',{'class':'unit_info'})[j].find_all('span',{'class':'txt_desc'})[1].get_text()
        )
        data2['time'].append(
            soup.find_all('div',{'class':'unit_info'})[j].find('span',{'class':'time_write'}).get_text()
        )
        data2['content'].append(
            soup.find_all('p', {'class':'txt_comment'})[j].get_text().strip('더보기')
        )

        style_attribute = soup.find_all('div', {'class':'star_info'})[j].find('span',{'class':'ico_star inner_star'})['style']
        width_percent = float(style_attribute.split(':')[1].strip('%;'))
        rating_out_of_five = width_percent / 20  
        data2['rating'].append(rating_out_of_five)

    df2 = pd.DataFrame(data2)
    df2.to_csv(f'./data2/{mer}.csv', index=False)

    try:
        with open(f'./image/{mer}1.png', 'wb') as f:
            f.write(image_data_decoded)

        with open(f'./image/{mer}2.png', 'wb') as f:
            f.write(image_data_decoded2)
    except:
        pass

    print(f"{mer}.csv 및 이미지 저장 완료")

    return data



In [2]:
def main():
    driver = webdriver.Chrome()
    driver.get('https://map.kakao.com/')
    time.sleep(2)

    searchbox = driver.find_element(By.ID, 'search.keyword.query')
    searchbox.send_keys('홍대 맛집'+ Keys.ENTER)
    time.sleep(3)

    list_url = []

    driver.find_element(By.ID, 'info.search.place.more').click()
    time.sleep(2)
    driver.find_element(By.ID, 'info.search.page.no1').click()
    time.sleep(2)

    for count in range(3):
        for inx in range(2, 6):
            moreview_element = driver.find_elements(By.CSS_SELECTOR, 'a[data-id="moreview"]')
            for i in range(len(moreview_element)):
                list_url.append(moreview_element[i].get_attribute('href'))
            driver.find_element(By.ID, f'info.search.page.no{inx}').click()
            if inx == 5:
                moreview_element = driver.find_elements(By.CSS_SELECTOR, 'a[data-id="moreview"]')
                for i in range(len(moreview_element)):
                    list_url.append(moreview_element[i].get_attribute('href'))
            time.sleep(2)
        driver.find_element(By.ID ,'info.search.page.next').click()
        time.sleep(2)
        
    print("음식점 개수 : ", len(list_url))

    driver.close()

    data_list = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(scrape_data, list_url)
        for result in results:
            data_list.append(result)

    df = pd.DataFrame(data_list)
    df.to_csv("./data1/data.csv", index=False)

if __name__ == "__main__":
    main()

음식점 개수 :  225
강강술래 홍대점 파싱완료
강강술래 홍대점.csv 및 이미지 저장 완료
또보겠지떡볶이집 깐따삐아점 파싱완료
또보겠지떡볶이집 깐따삐아점.csv 및 이미지 저장 완료
감성타코 홍대점 파싱완료
우와 홍대본점 파싱완료
맛이차이나 파싱완료
비스트로주라 파싱완료
카미야 파싱완료
땡스네이쳐 파싱완료
피오니 홍대점 파싱완료
칸다소바 홍대점 파싱완료
943킹스크로스 파싱완료
땡스네이쳐.csv 및 이미지 저장 완료
감성타코 홍대점.csv 및 이미지 저장 완료
홍대 조선시대 파싱완료
맛이차이나.csv 및 이미지 저장 완료
프리모바치오바치 홍대본점 파싱완료
하카타분코 파싱완료
우와 홍대본점.csv 및 이미지 저장 완료
카미야.csv 및 이미지 저장 완료
비스트로주라.csv 및 이미지 저장 완료
프리모바치오바치 홍대본점.csv 및 이미지 저장 완료
진진 파싱완료
테이스티버거 파싱완료
가미우동 파싱완료
진진.csv 및 이미지 저장 완료
츠케루 파싱완료
정돈 파싱완료
피오니 홍대점.csv 및 이미지 저장 완료
여우골 홍대점 파싱완료
칸다소바 홍대점.csv 및 이미지 저장 완료
하이디라오 홍대지점 파싱완료
마녀주방 홍대점 파싱완료
테이스티버거.csv 및 이미지 저장 완료
마녀주방 홍대점.csv 및 이미지 저장 완료
콜린 파싱완료
여우골 홍대점.csv 및 이미지 저장 완료
하카타분코.csv 및 이미지 저장 완료
공감 홍대2호점 파싱완료
공감 홍대2호점.csv 및 이미지 저장 완료
산더미불고기 파싱완료
콜린.csv 및 이미지 저장 완료
산더미불고기.csv 및 이미지 저장 완료
어반플랜트 합정 파싱완료
개화기요정 파싱완료
비스트로사랑방 파싱완료
하이디라오 홍대지점.csv 및 이미지 저장 완료
개화기요정.csv 및 이미지 저장 완료
비스트로사랑방.csv 및 이미지 저장 완료
더담 파싱완료
정돈.csv 및 이미지 저장 완료더담.csv 및 이미지 저장 완료

짬뽕지존 홍대점 파싱완료
이츠모라멘 파싱완료
이츠모라멘.csv 및 이미지 저장 완료
짬뽕지존 홍대점.csv 및 이미지 